# Bucket 분리 및 검수 워크플로우

## 워크플로우
1. JSON 파일 로드 → Bucket별 분리 (A, B, C)
2. 각 파일 검수 (A: 패스, B: 간단 검토, C: 정밀 검수)
3. 검수 완료 후 병합 → `text_raw` → `text_verified` 변환

## Step 0: Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 작업 디렉토리 설정 (본인 경로에 맞게 수정)
WORK_DIR = '/content/drive/MyDrive/TAPS'  # 수정 필요

import os
os.makedirs(WORK_DIR, exist_ok=True)

## Step 1: JSON 파일 로드 및 Bucket별 분리

In [ ]:
import json
from collections import defaultdict

def load_json(filepath):
    """JSON 파일 로드"""
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)

def save_json(data, filepath):
    """JSON 파일 저장"""
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"저장 완료: {filepath} ({len(data)}개 항목)")

def split_by_bucket(data):
    """Bucket별로 데이터 분리"""
    buckets = defaultdict(list)
    for item in data:
        bucket = item.get('bucket', 'unknown')
        buckets[bucket].append(item)
    return dict(buckets)

In [ ]:
# 원본 파일 경로 설정 (수정 필요)
TRAIN_FILE = f"{WORK_DIR}/asr_results_train_retriaged.json"
VAL_FILE = f"{WORK_DIR}/asr_results_validation_retriaged.json"  # 파일명 확인 필요

# 파일 로드
train_data = load_json(TRAIN_FILE)
print(f"Train 데이터: {len(train_data)}개")

# Validation 파일이 있으면 로드
try:
    val_data = load_json(VAL_FILE)
    print(f"Validation 데이터: {len(val_data)}개")
except FileNotFoundError:
    val_data = []
    print("Validation 파일 없음 (스킵)")

In [ ]:
# Bucket별 분리
train_buckets = split_by_bucket(train_data)
val_buckets = split_by_bucket(val_data) if val_data else {}

print("\n=== Train 분포 ===")
for bucket, items in sorted(train_buckets.items()):
    print(f"  Bucket {bucket}: {len(items)}개")

if val_buckets:
    print("\n=== Validation 분포 ===")
    for bucket, items in sorted(val_buckets.items()):
        print(f"  Bucket {bucket}: {len(items)}개")

In [ ]:
# Bucket별 파일 저장
OUTPUT_DIR = f"{WORK_DIR}/bucket_review"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Train 분리 저장
for bucket, items in train_buckets.items():
    save_json(items, f"{OUTPUT_DIR}/train_bucket_{bucket}.json")

# Validation 분리 저장
for bucket, items in val_buckets.items():
    save_json(items, f"{OUTPUT_DIR}/val_bucket_{bucket}.json")

print(f"\n모든 파일이 {OUTPUT_DIR}에 저장되었습니다.")

## Step 2: 검수 (수동 작업)

분리된 파일들을 검수합니다:
- **Bucket A**: 검수 불필요 (그대로 사용)
- **Bucket B**: 간단히 훑어보기 (필요시 `text_raw` 수정)
- **Bucket C**: 정밀 검수 (`text_raw` 직접 수정)

### 검수 방법
1. Google Drive에서 해당 JSON 파일 다운로드
2. 텍스트 에디터나 JSON 에디터로 열기
3. `text_raw` 필드 직접 수정
4. 저장 후 다시 업로드

또는 아래 셀에서 간단한 검수 도구 사용:

In [ ]:
# 간단한 검수 도구 (선택사항)
from IPython.display import display, clear_output
import ipywidgets as widgets

def review_bucket(filepath):
    """Bucket 파일 검수 도구"""
    data = load_json(filepath)
    current_idx = [0]  # mutable container for closure
    
    # Widgets
    output = widgets.Output()
    text_input = widgets.Textarea(
        description='text_raw:',
        layout=widgets.Layout(width='100%', height='100px')
    )
    idx_label = widgets.Label()
    
    def update_display():
        item = data[current_idx[0]]
        idx_label.value = f"[{current_idx[0]+1}/{len(data)}] {item['speaker_id']}_{item['sentence_id']}"
        text_input.value = item['text_raw']
        with output:
            clear_output()
            print(f"bucket: {item['bucket']} | reason: {item['reason']}")
            print(f"duration: {item['duration']:.2f}s | logprob: {item['avg_logprob']:.3f}")
    
    def save_current(_):
        data[current_idx[0]]['text_raw'] = text_input.value
        print("저장됨")
    
    def next_item(_):
        save_current(None)
        if current_idx[0] < len(data) - 1:
            current_idx[0] += 1
            update_display()
    
    def prev_item(_):
        save_current(None)
        if current_idx[0] > 0:
            current_idx[0] -= 1
            update_display()
    
    def save_file(_):
        save_current(None)
        save_json(data, filepath)
    
    # Buttons
    prev_btn = widgets.Button(description='◀ 이전')
    next_btn = widgets.Button(description='다음 ▶')
    save_btn = widgets.Button(description='💾 파일 저장', button_style='success')
    
    prev_btn.on_click(prev_item)
    next_btn.on_click(next_item)
    save_btn.on_click(save_file)
    
    update_display()
    
    display(widgets.VBox([
        idx_label,
        output,
        text_input,
        widgets.HBox([prev_btn, next_btn, save_btn])
    ]))

# 사용 예시 (검수할 파일 경로 입력)
# review_bucket(f"{OUTPUT_DIR}/train_bucket_C.json")

## Step 3: 검수 완료 후 병합

In [ ]:
def merge_and_convert(bucket_files, output_path):
    """
    Bucket 파일들을 병합하고 text_raw → text_verified로 변환
    """
    merged = []
    
    for filepath in bucket_files:
        if os.path.exists(filepath):
            data = load_json(filepath)
            merged.extend(data)
            print(f"로드: {filepath} ({len(data)}개)")
        else:
            print(f"파일 없음 (스킵): {filepath}")
    
    # text_raw → text_verified 변환
    for item in merged:
        item['text_verified'] = item.pop('text_raw')
    
    # index 재정렬 (선택사항)
    merged.sort(key=lambda x: (x['speaker_id'], x['sentence_id']))
    
    save_json(merged, output_path)
    return merged

In [ ]:
# Train 병합
train_bucket_files = [
    f"{OUTPUT_DIR}/train_bucket_A.json",
    f"{OUTPUT_DIR}/train_bucket_B.json",
    f"{OUTPUT_DIR}/train_bucket_C.json",
]

train_merged = merge_and_convert(
    train_bucket_files,
    f"{WORK_DIR}/train_verified.json"
)

print(f"\nTrain 병합 완료: {len(train_merged)}개")

In [ ]:
# Validation 병합 (있는 경우)
val_bucket_files = [
    f"{OUTPUT_DIR}/val_bucket_A.json",
    f"{OUTPUT_DIR}/val_bucket_B.json",
    f"{OUTPUT_DIR}/val_bucket_C.json",
]

# 파일이 하나라도 존재하면 병합
if any(os.path.exists(f) for f in val_bucket_files):
    val_merged = merge_and_convert(
        val_bucket_files,
        f"{WORK_DIR}/val_verified.json"
    )
    print(f"\nValidation 병합 완료: {len(val_merged)}개")
else:
    print("Validation bucket 파일 없음 (스킵)")

## Step 4: 결과 확인

In [ ]:
# 최종 결과 샘플 확인
final_train = load_json(f"{WORK_DIR}/train_verified.json")

print("=== 샘플 확인 ===")
for item in final_train[:3]:
    print(f"\n{item['speaker_id']}_{item['sentence_id']}:")
    print(f"  text_verified: {item['text_verified']}")
    print(f"  bucket: {item['bucket']}")

## 다음 단계

검수 완료된 `train_verified.json`, `val_verified.json` 파일을 normalizer에 적용:

```python
from taps.normalizer import normalize

for item in data:
    item['text_normalized'] = normalize(item['text_verified'])
```